<a href="https://colab.research.google.com/github/rauana-carvalho/Singularity/blob/main/Final_etl_singularity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ETL - LIMPEZA


In [ ]:
pip install fuzzywuzzy

In [ ]:
import pandas as pd
from google.colab import drive
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Montar Google Drive
drive.mount('/content/drive')

# Autenticar o usuário com credenciais padrão
creds, _ = default()
gc = gspread.authorize(creds)

def find_similar_columns(columns, threshold=80):
    """
    Encontra colunas semelhantes com base na similaridade de strings.

    Args:
    columns (list): Lista de nomes de colunas.
    threshold (int): Limite de similaridade para considerar duas colunas como semelhantes.

    Returns:
    dict: Dicionário com colunas semelhantes.
    """
    similar_cols_dict = {}
    processed = set()

    for col in columns:
        if col in processed:
            continue
        # Encontrar colunas semelhantes
        matches = process.extract(col, columns, limit=None)
        similar_cols = [match[0] for match in matches if match[1] >= threshold and match[0] != col]
        if similar_cols:
            similar_cols_dict[col] = similar_cols
            processed.update(similar_cols)  # Marcar como processadas

    return similar_cols_dict

def transform_text_to_dataframe(text):
    # A mesma função que você já tem
    lines = text.strip().split('\n')
    data = [line.split(',') for line in lines]

    max_columns = max([len(row) for row in data])
    for i in range(len(data)):
        if len(data[i]) < max_columns:
            data[i].extend([''] * (max_columns - len(data[i])))

    df = pd.DataFrame(data[1:], columns=data[0])
    df = df.loc[:, df.columns.str.strip() != '']
    unified_df = df.groupby(list(df.columns), as_index=False).first()
    return unified_df

def get_unique_or_existing_sheet(sheet, base_name):
    # A mesma função que você já tem
    existing_sheets = [ws.title for ws in sheet.worksheets()]
    if base_name in existing_sheets:
        worksheet = sheet.worksheet(base_name)
    else:
        new_name = base_name
        count = 1
        while new_name in existing_sheets:
            new_name = f"{base_name}_{count}"
            count += 1
        worksheet = sheet.add_worksheet(title=new_name, rows=1000, cols=26)
    return worksheet

def unify_similar_columns(df, similar_columns_dict):
    # A mesma função, mas adaptada para usar o dicionário de colunas semelhantes
    for col, similar_cols in similar_columns_dict.items():
        if len(similar_cols) > 0:
            for sim_col in similar_cols:
                if sim_col in df.columns:  # Verificar se a coluna existe
                    df[col] = df[col].combine_first(df[sim_col])
                    df = df.drop(columns=[sim_col])  # Remover a coluna similar após unir
    return df

def process_and_upload_to_sheets(folder_name, sheet_id):
    # Listar arquivos CSV na pasta
    folder_path = f"/content/drive/MyDrive/Singularity/{folder_name}/"
    files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    print("Arquivos encontrados:", files)

    # Abrir a planilha existente usando o ID
    sh = gc.open_by_key(sheet_id)
    print(f"Planilha com ID '{sheet_id}' já existe e foi aberta.")

    # Lista para armazenar DataFrames
    all_dataframes = []

    # Processar cada arquivo CSV
    for file in files:
        file_path = os.path.join(folder_path, file)

        # Ler o arquivo CSV
        df = pd.read_csv(file_path)
        print(f"Dados do arquivo '{file}' lidos:")
        print(df.head())

        # Limpar nomes de colunas
        df.columns = df.columns.str.strip()
        print("Colunas presentes no DataFrame:", df.columns)

        # Encontrar colunas semelhantes
        similar_columns_dict = find_similar_columns(df.columns.tolist(), threshold=80)
        print("Colunas semelhantes encontradas:", similar_columns_dict)

        # Unificar colunas similares
        df = unify_similar_columns(df, similar_columns_dict)
        print("Dados após unificação de colunas similares:")
        print(df.head())

        # Adicionar o DataFrame unificado à lista
        all_dataframes.append(df)

        # Obter ou criar a aba de acordo com o nome do arquivo
        base_name = file.split('.')[0]
        worksheet = get_unique_or_existing_sheet(sh, base_name)

        # Sobrescrever todos os dados, incluindo os cabeçalhos
        worksheet.clear()
        set_with_dataframe(worksheet, df, row=1, col=1, include_index=False)
        print(f"Dados enviados para a aba: {base_name}")

    # Depois de processar todas as abas, unificar os dados e colar na aba 'Base'
    base_worksheet = get_unique_or_existing_sheet(sh, "Base")

    # Concatenar todos os DataFrames em um único DataFrame
    final_df = pd.concat(all_dataframes, ignore_index=True)

    # Sobrescrever a aba 'Base'
    base_worksheet.clear()
    set_with_dataframe(base_worksheet, final_df, row=1, col=1, include_index=False)
    print("Dados unificados enviados para a aba 'Base'.")

# Nome da pasta do Google Drive onde os arquivos CSV estão armazenados
folder_name = 'Dados brutos'  # Substitua pelo nome da sua pasta

# ID da planilha no Google Sheets (extraído do link da planilha)
sheet_id = '1KI73nAPyJYTz6415SVZfDrCuJwoCJ_Gluv0LKJygCes'  # ID da planilha

# Executar o processo
process_and_upload_to_sheets(folder_name, sheet_id)


## ETL - UNIFICAÇÃO

In [ ]:
import pandas as pd
from google.colab import drive
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
import os
import re  # Para usar expressões regulares

# Montar Google Drive
drive.mount('/content/drive')

# Autenticar o usuário
auth.authenticate_user()

# Autorizar com as credenciais padrão do usuário (sem JSON)
creds, _ = default()
gc = gspread.authorize(creds)

def transform_text_to_dataframe(text):
    """
    Transformar texto em um DataFrame do pandas, onde o texto é dividido
    em colunas usando vírgula como delimitador. Campos iguais são unidos,
    mantendo apenas os primeiros valores.
    """
    lines = text.strip().split('\n')
    data = [line.split(',') for line in lines]

    # Ajuste para garantir que o número de colunas seja consistente
    max_columns = max([len(row) for row in data])

    # Preencher linhas que têm menos colunas com valores vazios
    for i in range(len(data)):
        if len(data[i]) < max_columns:
            data[i].extend([''] * (max_columns - len(data[i])))

    # Manter os cabeçalhos originais na primeira linha
    df = pd.DataFrame(data[1:], columns=data[0])

    # Remover colunas que tenham nomes vazios ou nulos
    df = df.loc[:, df.columns.str.strip() != '']

    # Agrupar por todas as colunas (removendo duplicatas)
    unified_df = df.groupby(list(df.columns), as_index=False).first()
    return unified_df

def get_unique_or_existing_sheet(sheet, base_name):
    """
    Retorna uma aba existente ou garante que o nome da nova aba seja único.
    """
    existing_sheets = [ws.title for ws in sheet.worksheets()]

    # Verificar se a aba com o nome base já existe
    if base_name in existing_sheets:
        print(f"Aba '{base_name}' já existe. Usando a aba existente.")
        worksheet = sheet.worksheet(base_name)
    else:
        # Se não existir, criar uma nova aba com um nome único
        new_name = base_name
        count = 1
        while new_name in existing_sheets:
            new_name = f"{base_name}_{count}"
            count += 1
        print(f"Criando nova aba: {new_name}")
        worksheet = sheet.add_worksheet(title=new_name, rows=1000, cols=26)  # Ajuste o número de linhas/colunas conforme necessário

    return worksheet

def unify_similar_columns(df, similar_columns_dict):
    """
    Une colunas similares em uma única coluna, mantendo os primeiros valores.
    """
    for col, similar_cols in similar_columns_dict.items():
        if len(similar_cols) > 1:  # Se houver mais de uma coluna similar
            for sim_col in similar_cols:
                if sim_col in df.columns and sim_col != col:  # Verificar se a coluna existe
                    df[col] = df[col].combine_first(df[sim_col])
                    df = df.drop(columns=[sim_col])  # Remover a coluna similar após unir
    return df

def remove_link_columns(df):
    """
    Remove colunas que contenham links (URLs) nos dados.
    """
    link_pattern = r"http[s]?://"
    cols_with_links = [col for col in df.columns if df[col].astype(str).str.contains(link_pattern).any()]

    if cols_with_links:
        print(f"Removendo colunas que contêm links: {cols_with_links}")
        df = df.drop(columns=cols_with_links)

    return df

def process_and_upload_to_sheets(folder_name, sheet_id):
    # Listar arquivos CSV na pasta
    folder_path = f"/content/drive/MyDrive/Singularity/{folder_name}/"
    files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    print("Arquivos encontrados:", files)

    # Abrir a planilha existente usando o ID
    sh = gc.open_by_key(sheet_id)
    print(f"Planilha com ID '{sheet_id}' já existe e foi aberta.")

    # Dicionário de colunas similares (substitua pelos nomes reais que você deseja unir)
    similar_columns_dict = {
        'Characteristics: Host Organism': ['Characteristics: Host Organism', 'Host Organism', 'Organism'],
        'Experiment Type': ['Experiment Type', 'Type of Experiment']
        # Adicione mais pares de colunas similares conforme necessário
    }

    # Lista para armazenar DataFrames
    all_dataframes = []

    # Processar cada arquivo CSV
    for file in files:
        file_path = os.path.join(folder_path, file)

        # Ler o arquivo CSV
        df = pd.read_csv(file_path)
        print(f"Dados do arquivo '{file}' lidos:")
        print(df.head())

        # Limpar e ajustar os nomes de colunas (substituir caracteres por underscores)
        df.columns = df.columns.str.replace(r'[.,;: ]+', '_', regex=True)

        # Verificar as colunas disponíveis
        print("Colunas presentes no DataFrame:", df.columns)

        # Adicionar uma coluna com o nome do arquivo
        df.insert(0, 'Source_File', file[:7])  # Adicionar os primeiros 7 dígitos do nome do arquivo

        # Transformar os dados em DataFrame unificado
        unified_df = transform_text_to_dataframe(df.to_csv(index=False))
        print(f"Dados unificados para o arquivo '{file}':")
        print(unified_df.head())

        # Unificar colunas similares
        unified_df = unify_similar_columns(unified_df, similar_columns_dict)
        print("Dados após unificação de colunas similares:")
        print(unified_df.head())

        # Remover colunas com links
        unified_df = remove_link_columns(unified_df)

        # Adicionar o DataFrame unificado à lista
        all_dataframes.append(unified_df)

        # Obter ou criar a aba de acordo com o nome do arquivo
        base_name = file.split('.')[0]
        worksheet = get_unique_or_existing_sheet(sh, base_name)

        # Sobrescrever todos os dados, incluindo os cabeçalhos
        worksheet.clear()

        # Colar os dados a partir da primeira linha
        set_with_dataframe(worksheet, unified_df, row=1, col=1, include_index=False)
        print(f"Dados enviados para a aba: {base_name}")

    # Depois de processar todas as abas, unificar os dados e colar na aba 'Base'
    base_worksheet = get_unique_or_existing_sheet(sh, "Base")

    # Concatenar todos os DataFrames em um único DataFrame
    final_df = pd.concat(all_dataframes, ignore_index=True)

    # Sobrescrever a aba 'Base'
    base_worksheet.clear()
    set_with_dataframe(base_worksheet, final_df, row=1, col=1, include_index=False)
    print("Dados unificados enviados para a aba 'Base'.")

# Nome da pasta do Google Drive onde os arquivos CSV estão armazenados
folder_name = 'Dados brutos'

# ID da planilha no Google Sheets (extraído do link da planilha)
sheet_id = '1KI73nAPyJYTz6415SVZfDrCuJwoCJ_Gluv0LKJygCes'

# Executar o processo
process_and_upload_to_sheets(folder_name, sheet_id)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arquivos encontrados: ['OSD-665-samples.csv', 'OSD-379-samples.csv']
Planilha com ID '1KI73nAPyJYTz6415SVZfDrCuJwoCJ_Gluv0LKJygCes' já existe e foi aberta.
Dados do arquivo 'OSD-665-samples.csv' lidos:
  Source Name                 Sample Name Characteristics: Organism  \
0    RR-23_F1  RR23_R-EDL_FLT_F1_techrep1              Mus musculus   
1    RR-23_F1  RR23_R-EDL_FLT_F1_techrep2              Mus musculus   
2    RR-23_F2           RR23_R-EDL_FLT_F2              Mus musculus   
3    RR-23_F3           RR23_R-EDL_FLT_F3              Mus musculus   
4    RR-23_F4           RR23_R-EDL_FLT_F4              Mus musculus   

  Characteristics: Strain Characteristics: Animal Source  \
0                C57BL/6J             Jackson Laboratory   
1                C57BL/6J             Jackson Laboratory   
2                C57BL/6J             Jackson Laboratory   
3 